In [78]:
import cv2
import numpy as np
import random
import os

In [79]:
def football_ground(image):
    #Convert image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    #Define lower and upper bounds for green color in HSV
    lower_green = np.array([40, 40, 40])  
    upper_green = np.array([70, 255, 255]) 
    

    mask = cv2.inRange(hsv_image, lower_green, upper_green)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = max(contours, key=cv2.contourArea)
    
    #Bouding box for largest contour
    x, y, w, h = cv2.boundingRect(largest_contour)
    #cv2.rectangle(image, (x+100, y), (x + (w-100), y + (h-120)), (0, 255, 0), 2)
    finalized_ground_area = [x+50, y+20, w-50, h]
    #finalized_ground_area = [x, y, w, h]
    return finalized_ground_area

In [80]:
def blur_bbox(image, bbox):
    x, y, w, h = bbox
    blurred_region = image[y:h, x:w]
    blurred_region = cv2.GaussianBlur(blurred_region, (101, 101), 0)
    image[y:h, x:w] = blurred_region
    return image

In [81]:
def change_ball(image):

    augmentation_techniques = [
    lambda img: cv2.flip(img, 1),                        # Horizontal flip
    lambda img: cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE), # Rotate
    lambda img: cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE), # Rotate
    #lambda img: cv2.blur(img, (5, 5)),                  # Blur
    #lambda img: cv2.cvtColor(img, cv2.COLOR_BGR2GRAY),  
    lambda img: cv2.resize(img, None, fx=1.1, fy=1.1) 
    #lambda img: cv2.threshold(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), 128, 255, cv2.THRESH_BINARY)[1] # Binarize
    ]

    random_technique = random.randint(0, len(augmentation_techniques) - 1)
    augmented_image = augmentation_techniques[random_technique](image)

    return augmented_image
    
    

In [82]:
def generate_ball_duplicates(image, ball_bbox, players_bboxes, ground_bbox, num_duplicates):
    image_height, image_width = image.shape[:2]
    ball_image = change_ball(ball_bbox)
    ball_height, ball_width = ball_image.shape[:2]
    duplicates = []
    bbox_coordinates = []

    while len(duplicates) < num_duplicates:
        x = random.randint(ground_bbox[0], ground_bbox[1] - ball_width)
        y = random.randint(ground_bbox[1], ground_bbox[3] - ball_height)
        ball_dup_bbox = (x, y)
        
        overlap = False
        for player_bbox in players_bboxes:
            if is_collision((x, y, x + ball_width, y + ball_height), player_bbox):
                overlap = True
                break

        if not overlap:
            duplicates.append(ball_dup_bbox)
            x_center = (x + (x+ball_width)) / 2 / image_width
            y_center = (y + (y+ball_height)) / 2 / image_height
            bbox_width = ((x+ball_width) - x) / image_width
            bbox_height = ((y+ball_height) - y) / image_height

            bbox_coordinates.append((x_center, y_center, bbox_width, bbox_height))
            image[y:y+ball_height, x:x+ball_width] = ball_image

    return image, bbox_coordinates

def is_collision(bbox1, bbox2):
    x1, y1, x2, y2 = bbox1
    x3, y3, x4, y4 = bbox2
    return not (x2 < x3 or x1 > x4 or y2 < y3 or y1 > y4)


In [83]:
images_path = "data/images"
labels_path = "data/labels"

images_out_path = "data_processed/images"
labels_out_path = "data_processed/labels"

In [84]:
for file_name in os.listdir(images_path):
    
    img = cv2.imread(os.path.join(images_path, file_name))
    
    label_file_name = os.path.splitext(file_name)[0] + '.txt'
    label_file_path = os.path.join(labels_path, label_file_name)
    
    img_width = img.shape[1]
    img_height = img.shape[0]
    img_bbox = img.copy()
    img_blur = img.copy()
    img_org = img.copy()
    
    ground_bbox = football_ground(img)
    
    ball_bbox = []
    players_bboxes = []
    with open(label_file_path, 'r') as labels:
        lines = labels.readlines()
        #print("lines.type: ", type(lines))
        #print("lines: ", lines)
        #print("lines: ",  lines)
        for line in lines:
            line_values = line.split(' ')
            if line_values[0] == '0':
                bbox = list(map(float, line_values[1:5]))  #float
                center_x, center_y, width, height = bbox
                
                xmin = int((center_x - width / 2) * img_width)
                ymin = int((center_y - height / 2) * img_height)
                xmax = int((center_x + width / 2) * img_width)
                ymax = int((center_y + height / 2) * img_height)
                bbox = [xmin, ymin, xmax, ymax]
                ball_bbox = bbox
                ball_image = img[ymin:ymax, xmin:xmax]
                
                
            else:
                bbox = list(map(float, line_values[1:5]))  #float
                center_x, center_y, width, height = bbox
                
                xmin = int((center_x - width / 2) * img_width)
                ymin = int((center_y - height / 2) * img_height)
                xmax = int((center_x + width / 2) * img_width)
                ymax = int((center_y + height / 2) * img_height)
                bbox = [xmin, ymin, xmax, ymax]
                players_bboxes.append(bbox)  
                #cv2.rectangle(img_bbox, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
                img_rslt = blur_bbox(img_blur, bbox)
        
        print("blured and info extracted...")
        num_duplicates = len(players_bboxes)
        print("num_duplicates: ", num_duplicates)
        img_temp = img_rslt.copy()
        img_duplicated, ball_cordinates = generate_ball_duplicates(img_org, ball_image, players_bboxes, ground_bbox, num_duplicates)
    
        for i, ball_cordinate in enumerate(ball_cordinates):
            if i == len(ball_cordinates) - 1:
                new_label = "0 "+str(ball_cordinate[0])+" "+str(ball_cordinate[1])+" "+str(ball_cordinate[2])+" "+str(ball_cordinate[3])+"\n" 
                lines.append(new_label)
            elif i == 0:
                new_label = "\n0 "+str(ball_cordinate[0])+" "+str(ball_cordinate[1])+" "+str(ball_cordinate[2])+" "+str(ball_cordinate[3])+"\n" 
                lines.append(new_label)
            else:
                new_label = "0 "+str(ball_cordinate[0])+" "+str(ball_cordinate[1])+" "+str(ball_cordinate[2])+" "+str(ball_cordinate[3])+"\n" 
                lines.append(new_label)
        #for i, ball_cordinate in enumerate(ball_cordinates):
        #   new_label =  "0 "+str(ball_cordinate[0])+" "+str(ball_cordinate[1])+" "+str(ball_cordinate[2])+" "+str(ball_cordinate[3])+"\n"
        #    lines.append(new_label)

        out_image_path = os.path.join(images_out_path, file_name)
        out_label_path = os.path.join(labels_out_path, label_file_name)
        
        with open(out_label_path, "w") as file:
            file.writelines(lines)

        cv2.imwrite( out_image_path,img_duplicated)
        

blured and info extracted...
num_duplicates:  24
blured and info extracted...
num_duplicates:  24
blured and info extracted...
num_duplicates:  24
blured and info extracted...
num_duplicates:  24
blured and info extracted...
num_duplicates:  22


ValueError: empty range for randrange() (476,181, -295)

In [85]:
###################################################################################################################################################